## Data Driver ADOBE

## Summary-subscription

In [1]:
import pandas as pd
from pymongo import MongoClient
import pymongo
from datetime import datetime, timedelta
import numpy as np

import csv

In [2]:
feh=datetime(2017,1,1,0,0,0)

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ecdf"]

In [4]:
mycol_cupAsig = mydb["cuponesasignados"]
def cuponesAsignadosdataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
cuponesAsignados = cuponesAsignadosdataframes(mycol_cupAsig.find({'createdAt': {'$gte':feh}},{'_id':1,'fecha_fin':1,'cupon_id':1,'fecha_activacion':1,'user_id_asignado':1,'createdAt':1 ,'producto_id':1,}), 10000)


In [5]:
cuponesAsignados.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11359 entries, 0 to 1358
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _id               11359 non-null  object        
 1   createdAt         11359 non-null  datetime64[ns]
 2   user_id_asignado  11359 non-null  object        
 3   cupon_id          11359 non-null  object        
 4   producto_id       11359 non-null  object        
 5   fecha_activacion  11359 non-null  datetime64[ns]
 6   fecha_fin         11359 non-null  datetime64[ns]
dtypes: datetime64[ns](3), object(4)
memory usage: 709.9+ KB


## Crucemos con pagos

In [6]:
mycol_pagos = mydb["pagos"]
def Pagosdataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
pagos = Pagosdataframes(mycol_pagos.find({'createdAt': {'$gte':feh}},{'_id':1,'monto':1,'acuerdo_cupon':1,'acuerdo_id':1,'moneda':1,'fecha_fin':1,'fecha_pago':1,'pasarela':1,'user_id':1,'createdAt':1 ,'boleta_id':1,'updatedAt':1 }), 10000)


In [7]:
pagos['mark']=pagos['_id']

In [8]:
pagos['mark'].nunique()

494646

pagos.info()

In [9]:
pagos['monto']=pagos['monto'].astype(float)

In [10]:
def dmitrypolo(row):
    if row["moneda"] == 'USD':
        return row['monto']*685
    else:
        return row['monto']

pagos["CLP_PRICE"] = pagos.apply(dmitrypolo, axis=1)

pagos

In [11]:
pagos['acuerdo_cupon']=pagos.acuerdo_cupon.astype(str)

In [12]:
cuponesAsignados['id']=cuponesAsignados._id.astype(str)

In [13]:
base_gateway=pagos.merge(cuponesAsignados, right_on='id',left_on='acuerdo_cupon',how='inner', validate='many_to_many')

In [14]:
base_gateway.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 713
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _id_x             714 non-null    object        
 1   updatedAt         714 non-null    datetime64[ns]
 2   createdAt_x       714 non-null    datetime64[ns]
 3   user_id           714 non-null    object        
 4   pasarela          714 non-null    object        
 5   monto             714 non-null    float64       
 6   moneda            714 non-null    object        
 7   fecha_pago        714 non-null    datetime64[ns]
 8   acuerdo_id        714 non-null    object        
 9   fecha_fin_x       714 non-null    datetime64[ns]
 10  boleta_id         638 non-null    object        
 11  acuerdo_cupon     714 non-null    object        
 12  mark              714 non-null    object        
 13  CLP_PRICE         714 non-null    float64       
 14  _id_y             714 non-

In [15]:
base_gateway

,_id_x,updatedAt,createdAt_x,user_id,pasarela,monto,moneda,fecha_pago,acuerdo_id,fecha_fin_x,...,mark,CLP_PRICE,_id_y,createdAt_y,user_id_asignado,cupon_id,producto_id,fecha_activacion,fecha_fin_y,id
0,5875788bbe13350cdd10fad7,2017-02-10 19:47:01.725,2017-01-11 00:12:59.286,563856cf38176617515ff5f3,transbank,7390.0,CLP,2017-01-11 00:12:59.282,5861ca9a0ee08c3630d82a9f,2017-02-23 00:12:59.282,...,5875788bbe13350cdd10fad7,7390.0,589e18b54320da55a2d1afbb,2017-02-10 19:47:01.723,563856cf38176617515ff5f3,589cd385cb67eb0a8e5630f0,5888c1b6f0d69c3b6b3c3c0c,2017-02-10 19:47:01.723,2017-02-23 00:12:59.282,589e18b54320da55a2d1afbb
1,5876f1642651911791c67232,2017-02-11 20:29:12.326,2017-01-12 03:00:52.509,563854d538176617515f358d,transbank,7390.0,CLP,2017-01-12 03:00:52.506,5861cf650ee08c3630da3d77,2017-02-24 03:00:52.506,...,5876f1642651911791c67232,7390.0,589f74184320da55a2d1c0a7,2017-02-11 20:29:12.325,563854d538176617515f358d,589cd385cb67eb0a8e5630f0,5888c1b6f0d69c3b6b3c3c0c,2017-02-11 20:29:12.324,2017-02-24 03:00:52.506,589f74184320da55a2d1c0a7
2,5876f1862651911791c6724c,2017-02-11 15:45:53.926,2017-01-12 03:01:26.550,5638570f3817661751600e9a,transbank,7390.0,CLP,2017-01-12 03:01:26.546,5861c6030ee08c3630d24a7c,2017-02-24 03:01:26.546,...,5876f1862651911791c6724c,7390.0,589f31b14320da55a2d1ba67,2017-02-11 15:45:53.925,5638570f3817661751600e9a,589cd385cb67eb0a8e5630f0,5888c1b6f0d69c3b6b3c3c0c,2017-02-11 15:45:53.923,2017-02-24 03:01:26.546,589f31b14320da55a2d1ba67
3,5876f18b2651911791c67250,2017-02-11 15:10:31.809,2017-01-12 03:01:31.757,563854e838176617515f3c9f,transbank,7390.0,CLP,2017-01-12 03:01:31.753,5861c4d70ee08c3630cecc3c,2017-02-24 03:01:31.753,...,5876f18b2651911791c67250,7390.0,589f29674320da55a2d1b9c9,2017-02-11 15:10:31.807,563854e838176617515f3c9f,589cd385cb67eb0a8e5630f0,5888c1b6f0d69c3b6b3c3c0c,2017-02-11 15:10:31.806,2017-02-24 03:01:31.753,589f29674320da55a2d1b9c9
4,5876f4f92651911791c672fc,2017-02-10 19:41:45.321,2017-01-12 03:16:09.858,5638559338176617515f7f17,transbank,7390.0,CLP,2017-01-12 03:16:09.854,5861c5330ee08c3630cfe1bd,2017-02-24 03:16:09.854,...,5876f4f92651911791c672fc,7390.0,589e17794320da55a2d1afb3,2017-02-10 19:41:45.319,5638559338176617515f7f17,589cd385cb67eb0a8e5630f0,5888c1b6f0d69c3b6b3c3c0c,2017-02-10 19:41:45.318,2017-02-24 03:16:09.854,589e17794320da55a2d1afb3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,5e4f4999edf4857ca98cb828,2020-04-10 11:54:21.081,2020-02-21 03:08:09.402,5b79870b9d27b50c17bef01f,transbank,10990.0,CLP,2020-02-21 03:08:09.398,5dfe5e6353b2ec0eea07e2a6,2020-10-16 23:59:59.999,...,5e4f4999edf4857ca98cb828,10990.0,5e67c3fef2bf150de08c6065,2020-03-10 16:44:46.221,5b79870b9d27b50c17bef01f,5e21c4d0d664b5687a8d8f7c,58530cb71652e44220a586d1,2020-03-10 16:44:46.221,2020-09-16 23:59:59.999,5e67c3fef2bf150de08c6065
710,5e5406cdd8e8fa176d15bbdb,2020-04-10 12:17:02.177,2020-02-24 17:24:29.334,5e5406bbd8e8fa176d15bbd8,cupon,131880.0,CLP,2020-02-24 17:24:29.330,5e2a0e8000da6a68745367b7,2022-03-15 23:59:59.999,...,5e5406cdd8e8fa176d15bbdb,131880.0,5e5d353dff4c640d9321fe8f,2020-03-02 16:33:01.099,5e5406bbd8e8fa176d15bbd8,5e2a0e8000da6a68745367b7,58530cb71652e44220a586d1,2020-03-02 16:33:01.097,2022-02-13 23:59:59.999,5e5d353dff4c640d9321fe8f
711,5e7ad7dfdae8ed1cbae78f90,2020-04-10 12:03:01.291,2020-03-25 04:02:39.675,5c8183eb1356387412cd46aa,transbank,10990.0,CLP,2020-03-25 04:02:39.672,5e51c719d8eff80dc944c27f,2021-05-19 23:59:59.999,...,5e7ad7dfdae8ed1cbae78f90,10990.0,5e836c63a7730b169fe6eeb6,2020-03-31 16:14:27.109,5c8183eb1356387412cd46aa,5e1f4807d664b5687a8d8f67,58530cb71652e44220a586d1,2020-03-31 16:14:27.108,2021-04-19 23:59:59.999,5e836c63a7730b169fe6eeb6
712,5e7c2a7d5bf37f0d8adf1661,2020-04-10 12:16:11.665,2020-03-26 04:07:25.900,581f3268e94abede08c265d3,cupon,10990.0,CLP,2020-03-26 04:07:25.897,5e600624d8e8fa176d15bbf4,2020-06-24 23:59:59.999,...,5e7c2a7d5bf37f0d8adf1661,10990.0,5e7c2aa65bf37f0d8adf1662,2020-03-26 04:08:06.702,581f326

In [16]:
base_gateway['SUBSCRIPTION_CODE']=base_gateway['acuerdo_id']

Ahora encontramos los pagos de los cupones de cortesía

In [17]:
pagos['acuerdo_id']=pagos.acuerdo_id.astype(str)

In [18]:
cuponesAsignados['cupon_id']=cuponesAsignados.cupon_id.astype(str)

In [19]:
base_cupon=pagos.merge(cuponesAsignados, right_on='cupon_id',left_on='acuerdo_id',how='inner', validate='many_to_many')

In [20]:
base_cupon.count()

_id_x               3657756
updatedAt           3657756
createdAt_x         3657756
user_id             3657756
pasarela            3657756
monto               3657756
moneda              3657756
fecha_pago          3657756
acuerdo_id          3657756
fecha_fin_x         3657756
boleta_id            109737
acuerdo_cupon       3657756
mark                3657756
CLP_PRICE           3657756
_id_y               3657756
createdAt_y         3657756
user_id_asignado    3657756
cupon_id            3657756
producto_id         3657756
fecha_activacion    3657756
fecha_fin_y         3657756
id                  3657756
dtype: int64

In [21]:
base_cupon['SUBSCRIPTION_CODE']=base_cupon['acuerdo_id']

In [22]:
consolida = [base_cupon, base_gateway]

In [23]:
cupones_totales=pd.concat(consolida)

cupones_totales.info()

cupones_totales.count()

In [24]:
cupones_totales['ACTIVE_SUB_END_DATE']=cupones_totales['fecha_fin_x']

In [25]:
cupones_totales['ACTIVE_SUB_START_DATE']=cupones_totales['fecha_activacion']

In [26]:
cupones_totales['ACTIVE_SUB_LAST_PAYMENT_DATE']=cupones_totales['fecha_pago']

In [27]:
cupones_totales.columns

Index(['_id_x', 'updatedAt', 'createdAt_x', 'user_id', 'pasarela', 'monto',
       'moneda', 'fecha_pago', 'acuerdo_id', 'fecha_fin_x', 'boleta_id',
       'acuerdo_cupon', 'mark', 'CLP_PRICE', '_id_y', 'createdAt_y',
       'user_id_asignado', 'cupon_id', 'producto_id', 'fecha_activacion',
       'fecha_fin_y', 'id', 'SUBSCRIPTION_CODE', 'ACTIVE_SUB_END_DATE',
       'ACTIVE_SUB_START_DATE', 'ACTIVE_SUB_LAST_PAYMENT_DATE'],
      dtype='object')

cupones_totales.columns

In [28]:
cupones_totales=cupones_totales.drop([ '_id_x', 'updatedAt', 'createdAt_x', 'pasarela', 'monto',
       'moneda', 'fecha_pago', 'acuerdo_id', 'fecha_fin_x', 'boleta_id',
       'acuerdo_cupon',  '_id_y', 'createdAt_y',
       'user_id_asignado', 'cupon_id', 'producto_id', 'fecha_activacion',
       'fecha_fin_y', 'id'],axis=1)

In [29]:
cupones_totales.count()

user_id                         3658470
mark                            3658470
CLP_PRICE                       3658470
SUBSCRIPTION_CODE               3658470
ACTIVE_SUB_END_DATE             3658470
ACTIVE_SUB_START_DATE           3658470
ACTIVE_SUB_LAST_PAYMENT_DATE    3658470
dtype: int64

from operator import attrgetter 
cupones_totales['DURATION']=(cupones_totales.START_DATE.dt.to_period('D') - cupones_totales.END_DATE.dt.to_period('D')).apply(attrgetter('n')).abs()

cupones_totales['START_DATE']=cupones_totales['START_DATE'] .map(lambda x: str(x)[:10])

In [30]:
mycol_users = mydb["users"]
def usersdataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
users = usersdataframes(mycol_users.find({},{'_id':1,'_cliente':1}), 10000)


In [31]:
users['user_id']=users._id.astype(str)

In [32]:
cupones_totales

,user_id,mark,CLP_PRICE,SUBSCRIPTION_CODE,ACTIVE_SUB_END_DATE,ACTIVE_SUB_START_DATE,ACTIVE_SUB_LAST_PAYMENT_DATE
0,56385830381766175160756f,586d53e86bf40d2fd7c89e87,0.0,5865287270b7325b5207da59,2017-02-03 00:00:00.000,2017-01-04 19:58:32.374,2017-01-04 19:58:32.374
1,56385830381766175160756f,586d53e86bf40d2fd7c89e87,0.0,5865287270b7325b5207da59,2017-02-03 00:00:00.000,2017-01-09 15:52:09.683,2017-01-04 19:58:32.374
2,56385830381766175160756f,586d53e86bf40d2fd7c89e87,0.0,5865287270b7325b5207da59,2017-02-03 00:00:00.000,2017-01-11 18:15:28.156,2017-01-04 19:58:32.374
3,56385830381766175160756f,586d53e86bf40d2fd7c89e87,0.0,5865287270b7325b5207da59,2017-02-03 00:00:00.000,2017-01-11 22:21:48.015,2017-01-04 19:58:32.374
4,56385830381766175160756f,586d53e86bf40d2fd7c89e87,0.0,5865287270b7325b5207da59,2017-02-03 00:00:00.000,2017-02-17 15:01:40.978,2017-01-04 19:58:32.374
...,...,...,...,...,...,...,...
709,5b79870b9d27b50c17bef01f,5e4f4999edf4857ca98cb828,10990.0,5dfe5e6353b2ec0eea07e2a6,2020-10-16 23:59:59.999,2020-03-10 16:44:46.221,2020-02-21 03:08:09.398
710,5e5406bbd8e8fa176d15bbd8,5e5406cdd8e8fa176d15bbdb,131880.0,5e2a0e8000da6a68745367b7,2022-03-15 23:59:59.999,2020-03-02 16:33:01.097,2020-02-24 17:24:29.330
711,5c8183eb1356387412cd46aa,5e7ad7dfdae8ed1cbae78f90,10990.0,5e51c719d8eff80dc944c27f,2021-05-19 23:59:59.999,2020-03-31 16:14:27.108,2020-03-25 04:02:39.672
712,581f3268e94abede08c265d3,5e7c2a7d5bf37f0d8adf1661,10990.0,5e600624d8e8fa176d15bbf4,2020-06-24 23:59:59.999,2020-03-26 04:08:06.701,2020-03-26 04:07:25.897


In [33]:
cupones_totales=cupones_totales.merge(users, on='user_id',how='left')

In [34]:
cupones_totales.count()

user_id                         3658470
mark                            3658470
CLP_PRICE                       3658470
SUBSCRIPTION_CODE               3658470
ACTIVE_SUB_END_DATE             3658470
ACTIVE_SUB_START_DATE           3658470
ACTIVE_SUB_LAST_PAYMENT_DATE    3658470
_id                             3658445
_cliente                        3658445
dtype: int64

In [35]:
cupones_totales=cupones_totales.rename(columns={ "_cliente":"CUSTOMER_ID"})

In [36]:
cupones_totales = cupones_totales.dropna(axis=0, subset=['user_id','_id'])

## Añadir transbank y paypal

In [37]:
mycol_paypalAsig= mydb["acuerdospaypals"]
def paypalAsigframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
paypal = paypalAsigframes(mycol_paypalAsig.find({'createdAt': {'$gte':feh}},{'_id':1,'createdAt':1,'user_id':1}), 10000)

In [38]:
paypal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31695 entries, 0 to 1694
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   _id        31695 non-null  object        
 1   createdAt  31695 non-null  datetime64[ns]
 2   user_id    31695 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 990.5+ KB


In [39]:
paypal['user_id']=paypal.user_id.astype(str)

In [40]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434819 entries, 0 to 4818
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   _id       434819 non-null  object
 1   _cliente  434759 non-null  object
 2   user_id   434819 non-null  object
dtypes: object(3)
memory usage: 13.3+ MB


In [41]:
users['id']=users._id.astype(str)

In [42]:
user_Paypal=users.merge(paypal, right_on='user_id',left_on='id',how='right')

In [43]:
user_Paypal = user_Paypal.dropna(axis=0, subset=['_id_x'])

In [44]:
user_Paypal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31690 entries, 0 to 31689
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   _id_x      31690 non-null  object        
 1   _cliente   31690 non-null  object        
 2   user_id_x  31690 non-null  object        
 3   id         31690 non-null  object        
 4   _id_y      31690 non-null  object        
 5   createdAt  31690 non-null  datetime64[ns]
 6   user_id_y  31690 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 1.9+ MB


In [45]:
user_Paypal=user_Paypal.rename(columns={ "_cliente":"CUSTOMER_ID","_id_y":"SUBSCRIPTION_CODE"})

In [46]:
pagos.columns

Index(['_id', 'updatedAt', 'createdAt', 'user_id', 'pasarela', 'monto',
       'moneda', 'fecha_pago', 'acuerdo_id', 'fecha_fin', 'boleta_id',
       'acuerdo_cupon', 'mark', 'CLP_PRICE'],
      dtype='object')

In [47]:
user_Paypal['SUBSCRIPTION_CODE']=user_Paypal.SUBSCRIPTION_CODE.astype(str)

In [48]:
userPag_Paypal=user_Paypal.merge(pagos, right_on='acuerdo_id',left_on='SUBSCRIPTION_CODE',how='left')

In [49]:
userPag_Paypal = userPag_Paypal.dropna(axis=0, subset=['_id'])

In [50]:
userPag_Paypal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39949 entries, 0 to 63199
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   _id_x              39949 non-null  object        
 1   CUSTOMER_ID        39949 non-null  object        
 2   user_id_x          39949 non-null  object        
 3   id                 39949 non-null  object        
 4   SUBSCRIPTION_CODE  39949 non-null  object        
 5   createdAt_x        39949 non-null  datetime64[ns]
 6   user_id_y          39949 non-null  object        
 7   _id                39949 non-null  object        
 8   updatedAt          39949 non-null  datetime64[ns]
 9   createdAt_y        39949 non-null  datetime64[ns]
 10  user_id            39949 non-null  object        
 11  pasarela           39949 non-null  object        
 12  monto              39949 non-null  float64       
 13  moneda             39949 non-null  object        
 14  fecha_

In [51]:
userPag_Paypal['ACTIVE_SUB_LAST_PAYMENT_DATE'] = pd.to_datetime(userPag_Paypal['fecha_pago'])

In [52]:
userPag_Paypal['ACTIVE_SUB_END_DATE'] = pd.to_datetime(userPag_Paypal['fecha_fin'])

In [53]:
userPag_Paypal['ACTIVE_SUB_START_DATE'] = pd.to_datetime(userPag_Paypal['createdAt_x'])

In [54]:
userPag_Paypal['SUBSCRIPTION_CODE']=userPag_Paypal['acuerdo_id']

In [55]:
userPag_Paypal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39949 entries, 0 to 63199
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   _id_x                         39949 non-null  object        
 1   CUSTOMER_ID                   39949 non-null  object        
 2   user_id_x                     39949 non-null  object        
 3   id                            39949 non-null  object        
 4   SUBSCRIPTION_CODE             39949 non-null  object        
 5   createdAt_x                   39949 non-null  datetime64[ns]
 6   user_id_y                     39949 non-null  object        
 7   _id                           39949 non-null  object        
 8   updatedAt                     39949 non-null  datetime64[ns]
 9   createdAt_y                   39949 non-null  datetime64[ns]
 10  user_id                       39949 non-null  object        
 11  pasarela                    

In [56]:
userPag_Paypal.columns

Index(['_id_x', 'CUSTOMER_ID', 'user_id_x', 'id', 'SUBSCRIPTION_CODE',
       'createdAt_x', 'user_id_y', '_id', 'updatedAt', 'createdAt_y',
       'user_id', 'pasarela', 'monto', 'moneda', 'fecha_pago', 'acuerdo_id',
       'fecha_fin', 'boleta_id', 'acuerdo_cupon', 'mark', 'CLP_PRICE',
       'ACTIVE_SUB_LAST_PAYMENT_DATE', 'ACTIVE_SUB_END_DATE',
       'ACTIVE_SUB_START_DATE'],
      dtype='object')

In [57]:
userPag_Paypal=userPag_Paypal.drop(['_id_x','user_id_x', 'id', 
       'createdAt_x', 'user_id_y', '_id', 'updatedAt',
       'createdAt_y', 'user_id', 'pasarela', 'monto', 'moneda', 'fecha_pago',
       'acuerdo_id', 'fecha_fin', 'boleta_id', 'acuerdo_cupon'],axis=1)

In [58]:
userPag_Paypal.columns

Index(['CUSTOMER_ID', 'SUBSCRIPTION_CODE', 'mark', 'CLP_PRICE',
       'ACTIVE_SUB_LAST_PAYMENT_DATE', 'ACTIVE_SUB_END_DATE',
       'ACTIVE_SUB_START_DATE'],
      dtype='object')

from operator import attrgetter 
userPag_Paypal['DURATION']=(userPag_Paypal.START_DATE.dt.to_period('D') - userPag_Paypal.END_DATE.dt.to_period('D')).apply(attrgetter('n')).abs()

In [59]:
consol=[userPag_Paypal,cupones_totales]

In [60]:
historyParcial=pd.concat(consol)

In [61]:
historyParcial.count()

CUSTOMER_ID                     3698394
SUBSCRIPTION_CODE               3698394
mark                            3698394
CLP_PRICE                       3698394
ACTIVE_SUB_LAST_PAYMENT_DATE    3698394
ACTIVE_SUB_END_DATE             3698394
ACTIVE_SUB_START_DATE           3698394
user_id                         3658445
_id                             3658445
dtype: int64

In [62]:
mycol_transAsig = mydb["transbankfinalizaregistros"]
def transAsigframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
transbank = transAsigframes(mycol_transAsig.find({},{'_id':1,'createdAt':1,'user_id':1}), 10000)

In [63]:
transbank=transbank.rename(columns={"_id":"SUBSCRIPTION_CODE"})

In [64]:
transbank['user_id']=transbank.user_id.astype(str)

In [65]:
transbank['SUBSCRIPTION_CODE']=transbank.SUBSCRIPTION_CODE.astype(str)

In [66]:
users.columns

Index(['_id', '_cliente', 'user_id', 'id'], dtype='object')

In [67]:
user_transbank=users.merge(transbank, right_on='user_id',left_on='id',how='right')

In [68]:
user_transbank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135371 entries, 0 to 135370
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id                135367 non-null  object        
 1   _cliente           135367 non-null  object        
 2   user_id_x          135367 non-null  object        
 3   id                 135367 non-null  object        
 4   SUBSCRIPTION_CODE  135371 non-null  object        
 5   createdAt          135371 non-null  datetime64[ns]
 6   user_id_y          135371 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 8.3+ MB


In [69]:
user_transbank = user_transbank.dropna(axis=0, subset=['_id'])

In [70]:
user_transbank=user_transbank.rename(columns={ "_cliente":"CUSTOMER_ID"})

In [71]:
user_transbank['START_DATE']=user_transbank['createdAt']

In [72]:
pagos.columns

Index(['_id', 'updatedAt', 'createdAt', 'user_id', 'pasarela', 'monto',
       'moneda', 'fecha_pago', 'acuerdo_id', 'fecha_fin', 'boleta_id',
       'acuerdo_cupon', 'mark', 'CLP_PRICE'],
      dtype='object')

In [73]:
user_Pagtransbank=user_transbank.merge(pagos, right_on='acuerdo_id',left_on='SUBSCRIPTION_CODE',how='left')

In [74]:
user_Pagtransbank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486513 entries, 0 to 486512
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id_x              486513 non-null  object        
 1   CUSTOMER_ID        486513 non-null  object        
 2   user_id_x          486513 non-null  object        
 3   id                 486513 non-null  object        
 4   SUBSCRIPTION_CODE  486513 non-null  object        
 5   createdAt_x        486513 non-null  datetime64[ns]
 6   user_id_y          486513 non-null  object        
 7   START_DATE         486513 non-null  datetime64[ns]
 8   _id_y              445696 non-null  object        
 9   updatedAt          445696 non-null  datetime64[ns]
 10  createdAt_y        445696 non-null  datetime64[ns]
 11  user_id            445696 non-null  object        
 12  pasarela           445696 non-null  object        
 13  monto              445696 non-null  float64 

In [75]:
user_Pagtransbank = user_Pagtransbank.dropna(axis=0, subset=['CLP_PRICE'])

In [76]:
user_Pagtransbank['ACTIVE_SUB_LAST_PAYMENT_DATE'] = pd.to_datetime(user_Pagtransbank['fecha_pago'])

In [77]:
user_Pagtransbank['ACTIVE_SUB_END_DATE'] = pd.to_datetime(user_Pagtransbank['fecha_fin'])

from operator import attrgetter 
user_Pagtransbank['DURATION']=(user_Pagtransbank.START_DATE.dt.to_period('D') - user_Pagtransbank.END_DATE.dt.to_period('D')).apply(attrgetter('n')).abs()

In [78]:
user_Pagtransbank['ACTIVE_SUB_START_DATE']=user_Pagtransbank['START_DATE'] 

In [79]:
user_Pagtransbank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445696 entries, 3 to 486512
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   _id_x                         445696 non-null  object        
 1   CUSTOMER_ID                   445696 non-null  object        
 2   user_id_x                     445696 non-null  object        
 3   id                            445696 non-null  object        
 4   SUBSCRIPTION_CODE             445696 non-null  object        
 5   createdAt_x                   445696 non-null  datetime64[ns]
 6   user_id_y                     445696 non-null  object        
 7   START_DATE                    445696 non-null  datetime64[ns]
 8   _id_y                         445696 non-null  object        
 9   updatedAt                     445696 non-null  datetime64[ns]
 10  createdAt_y                   445696 non-null  datetime64[ns]
 11  user_id      

In [80]:
user_Pagtransbank.columns

Index(['_id_x', 'CUSTOMER_ID', 'user_id_x', 'id', 'SUBSCRIPTION_CODE',
       'createdAt_x', 'user_id_y', 'START_DATE', '_id_y', 'updatedAt',
       'createdAt_y', 'user_id', 'pasarela', 'monto', 'moneda', 'fecha_pago',
       'acuerdo_id', 'fecha_fin', 'boleta_id', 'acuerdo_cupon', 'mark',
       'CLP_PRICE', 'ACTIVE_SUB_LAST_PAYMENT_DATE', 'ACTIVE_SUB_END_DATE',
       'ACTIVE_SUB_START_DATE'],
      dtype='object')

In [81]:
user_Pagtransbank=user_Pagtransbank.drop(['_id_x', 'id', 'createdAt_x',
       'user_id_x', '_id_y', 'updatedAt', 'user_id', 'createdAt_y',
       'user_id_y', 'pasarela', 'monto', 'moneda', 'fecha_pago', 'acuerdo_id',
       'fecha_fin', 'boleta_id', 'acuerdo_cupon'],axis=1)

In [82]:
user_Pagtransbank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445696 entries, 3 to 486512
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   CUSTOMER_ID                   445696 non-null  object        
 1   SUBSCRIPTION_CODE             445696 non-null  object        
 2   START_DATE                    445696 non-null  datetime64[ns]
 3   mark                          445696 non-null  object        
 4   CLP_PRICE                     445696 non-null  float64       
 5   ACTIVE_SUB_LAST_PAYMENT_DATE  445696 non-null  datetime64[ns]
 6   ACTIVE_SUB_END_DATE           445696 non-null  datetime64[ns]
 7   ACTIVE_SUB_START_DATE         445696 non-null  datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(3)
memory usage: 30.6+ MB


In [83]:
consolidado=[historyParcial,user_Pagtransbank]

In [84]:
history=pd.concat(consolidado)


In [85]:
history.count()

CUSTOMER_ID                     4144090
SUBSCRIPTION_CODE               4144090
mark                            4144090
CLP_PRICE                       4144090
ACTIVE_SUB_LAST_PAYMENT_DATE    4144090
ACTIVE_SUB_END_DATE             4144090
ACTIVE_SUB_START_DATE           4144090
user_id                         3658445
_id                             3658445
START_DATE                       445696
dtype: int64

Muy bien, con este resultado:
    1. Obtener SUBSCRIPTIONS_LIFETIME y REVENUE_LIFETIME

Entonces se tienen que sacar las cuentas:
    Para extraer el data set de los acuerdos activos ( por last_pyment)
    Luego obtener ACTIVE_SUB_DURATION  y HAS_ACTIVE_SUBSCRIPTION

Seguido se debe consultar:
    Acuerdos cancelados para obtener CANCELLATIONS_LIFETIME
Aquí podemos verificar que SUBSCRIPTIONS_LIFETIME - CANCELLATIONS_LIFETIME sea positivo

    

In [86]:
history['ACTIVE_SUB_LAST_PAYMENT_DATE'] = pd.to_datetime(history['ACTIVE_SUB_LAST_PAYMENT_DATE'])

In [87]:
import datetime

In [88]:
Time_Line=datetime.datetime.today()

In [89]:
history_SubsActives=history.loc[history['ACTIVE_SUB_END_DATE'] > Time_Line]

In [90]:
history_SubsActives.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35134 entries, 20 to 486512
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   CUSTOMER_ID                   35134 non-null  object        
 1   SUBSCRIPTION_CODE             35134 non-null  object        
 2   mark                          35134 non-null  object        
 3   CLP_PRICE                     35134 non-null  float64       
 4   ACTIVE_SUB_LAST_PAYMENT_DATE  35134 non-null  datetime64[ns]
 5   ACTIVE_SUB_END_DATE           35134 non-null  datetime64[ns]
 6   ACTIVE_SUB_START_DATE         35134 non-null  datetime64[ns]
 7   user_id                       26933 non-null  object        
 8   _id                           26933 non-null  object        
 9   START_DATE                    7484 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(5)
memory usage: 2.9+ MB


In [91]:
subscriptions_actives=history_SubsActives.sort_values('ACTIVE_SUB_LAST_PAYMENT_DATE', ascending=False).groupby('CUSTOMER_ID').tail(1)

In [92]:
subscriptions_actives.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8664 entries, 26465 to 629
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   CUSTOMER_ID                   8664 non-null   object        
 1   SUBSCRIPTION_CODE             8664 non-null   object        
 2   mark                          8664 non-null   object        
 3   CLP_PRICE                     8664 non-null   float64       
 4   ACTIVE_SUB_LAST_PAYMENT_DATE  8664 non-null   datetime64[ns]
 5   ACTIVE_SUB_END_DATE           8664 non-null   datetime64[ns]
 6   ACTIVE_SUB_START_DATE         8664 non-null   datetime64[ns]
 7   user_id                       472 non-null    object        
 8   _id                           472 non-null    object        
 9   START_DATE                    7481 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(5)
memory usage: 744.6+ KB


In [93]:
subscriptions_actives=subscriptions_actives.drop([ 'mark','user_id','_id','START_DATE'],axis=1)

In [94]:
subscriptions_actives['HAS_ACTIVE_SUBSCRIPTION']=1

In [95]:
diff=(subscriptions_actives['ACTIVE_SUB_END_DATE']- subscriptions_actives['ACTIVE_SUB_START_DATE'])
subscriptions_actives['ACTIVE_SUB_DURATION']=(diff.astype('timedelta64[D]')).astype(int)


In [96]:
subscriptions_actives

,CUSTOMER_ID,SUBSCRIPTION_CODE,CLP_PRICE,ACTIVE_SUB_LAST_PAYMENT_DATE,ACTIVE_SUB_END_DATE,ACTIVE_SUB_START_DATE,HAS_ACTIVE_SUBSCRIPTION,ACTIVE_SUB_DURATION
26465,586101b804d5b52eef205079,5e30ae485f2f260deee70dff,11645.00,2020-04-27 14:43:05.000,2020-05-27 23:59:59.999,2020-01-28 21:57:28.073,1,120
11471,5861007204d5b52eef1f519f,5bdf2c2539d6040b065590f0,11645.00,2020-04-27 13:12:33.000,2020-05-28 23:59:59.999,2018-11-04 17:28:05.302,1,571
30573,58a07cac4320da55a2d1d90c,58a07d334320da55a2d1d942,10610.65,2020-04-27 12:42:02.000,2020-05-27 23:59:59.999,2017-02-12 15:20:19.707,1,1200
62989,5e5835dc7629f813181aed11,5e5837577629f813181aed21,11645.00,2020-04-27 12:27:22.000,2020-05-27 23:59:59.999,2020-02-27 21:40:39.034,1,90
28373,5861020304d5b52eef208b20,5ac9d21837b6490ad3bbcd46,10610.65,2020-04-27 11:46:04.000,2020-05-27 23:59:59.999,2018-04-08 08:26:00.020,1,780
...,...,...,...,...,...,...,...,...
976,5910a3b1048691707c146fc6,58644bbc7064005a9124ca62,0.00,2017-05-08 19:27:42.396,2021-01-30 23:59:59.999,2018-06-11 20:45:20.182,1,964
23825,5861015e04d5b52eef200b4e,5866a8eb3c27ec7219b7aab5,0.00,2017-02-16 18:53:57.489,2020-12-31 00:00:00.000,2017-02-22 19:33:10.209,1,1407
861,587f8855c3fc85671876d63a,58644bbc7064005a9124ca62,0.00,2017-01-24 17:58:24.757,2021-01-30 23:59:59.999,2018-08-22 20:14:55.695,1,892
739,586101b304d5b52eef204c33,58644bbc7064005a9124ca62,0.00,2017-01-17 17:08:38.574,2021-01-30 23:59:59.999,2018-08-24 14:17:44.402,1,890


In [97]:
history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4144090 entries, 0 to 486512
Data columns (total 10 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   CUSTOMER_ID                   object        
 1   SUBSCRIPTION_CODE             object        
 2   mark                          object        
 3   CLP_PRICE                     float64       
 4   ACTIVE_SUB_LAST_PAYMENT_DATE  datetime64[ns]
 5   ACTIVE_SUB_END_DATE           datetime64[ns]
 6   ACTIVE_SUB_START_DATE         datetime64[ns]
 7   user_id                       object        
 8   _id                           object        
 9   START_DATE                    datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(5)
memory usage: 347.8+ MB


In [98]:
paysLong=history.groupby('CUSTOMER_ID').agg(REVENUE_LIFETIME=('CLP_PRICE','sum'))

In [99]:
paysLong.count()

REVENUE_LIFETIME    66416
dtype: int64

In [100]:
paysLong['CUSTOMER_ID']=paysLong.index

In [101]:
paysLong=paysLong.reset_index(drop=True)

In [102]:
paysLong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66416 entries, 0 to 66415
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REVENUE_LIFETIME  66416 non-null  float64
 1   CUSTOMER_ID       66416 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.0+ MB


In [103]:
paysLong

,REVENUE_LIFETIME,CUSTOMER_ID
0,131880.0,5860ff4104d5b52eef1e6224
1,29560.0,5860ff4104d5b52eef1e622f
2,98910.0,5860ff4104d5b52eef1e6243
3,30772.0,5860ff4104d5b52eef1e6250
4,109900.0,5860ff4104d5b52eef1e6264
...,...,...
66411,10990.0,5e9df0188a922b726c7b6752
66412,10990.0,5e9e3e5a4099310f332f1411
66413,10990.0,5e9e47fbf1106b0f0744fbda
66414,11645.0,5ea0b295a796be1213975b04


In [104]:
preresultado=subscriptions_actives.merge(paysLong, on='CUSTOMER_ID',how='outer')

In [105]:
preresultado["HAS_ACTIVE_SUBSCRIPTION"] = preresultado["HAS_ACTIVE_SUBSCRIPTION"].fillna(0).astype(int)

In [106]:
preresultado

,CUSTOMER_ID,SUBSCRIPTION_CODE,CLP_PRICE,ACTIVE_SUB_LAST_PAYMENT_DATE,ACTIVE_SUB_END_DATE,ACTIVE_SUB_START_DATE,HAS_ACTIVE_SUBSCRIPTION,ACTIVE_SUB_DURATION,REVENUE_LIFETIME
0,586101b804d5b52eef205079,5e30ae485f2f260deee70dff,11645.00,2020-04-27 14:43:05,2020-05-27 23:59:59.999,2020-01-28 21:57:28.073,1,120.0,285885.0
1,5861007204d5b52eef1f519f,5bdf2c2539d6040b065590f0,11645.00,2020-04-27 13:12:33,2020-05-28 23:59:59.999,2018-11-04 17:28:05.302,1,571.0,395930.0
2,58a07cac4320da55a2d1d90c,58a07d334320da55a2d1d942,10610.65,2020-04-27 12:42:02,2020-05-27 23:59:59.999,2017-02-12 15:20:19.707,1,1200.0,424426.0
3,5e5835dc7629f813181aed11,5e5837577629f813181aed21,11645.00,2020-04-27 12:27:22,2020-05-27 23:59:59.999,2020-02-27 21:40:39.034,1,90.0,34935.0
4,5861020304d5b52eef208b20,5ac9d21837b6490ad3bbcd46,10610.65,2020-04-27 11:46:04,2020-05-27 23:59:59.999,2018-04-08 08:26:00.020,1,780.0,424426.0
...,...,...,...,...,...,...,...,...,...
66411,5e6ebac1faadb50d9386c4d8,NaN,NaN,NaT,NaT,NaT,0,NaN,11645.0
66412,5e6ebb3b7629f813181b8d9c,NaN,NaN,NaT,NaT,NaT,0,NaN,10990.0
66413,5e6eee52ad807c1304b5c100,NaN,NaN,NaT,NaT,NaT,0,NaN,10990.0
66414,5e713a5b3687525ad4328dbe,NaN,NaN,NaT,NaT,NaT,0,NaN,10990.0


Ahora buscar las subscriptions lifetime

In [107]:
cupones_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3658445 entries, 0 to 3658469
Data columns (total 9 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   user_id                       object        
 1   mark                          object        
 2   CLP_PRICE                     float64       
 3   SUBSCRIPTION_CODE             object        
 4   ACTIVE_SUB_END_DATE           datetime64[ns]
 5   ACTIVE_SUB_START_DATE         datetime64[ns]
 6   ACTIVE_SUB_LAST_PAYMENT_DATE  datetime64[ns]
 7   _id                           object        
 8   CUSTOMER_ID                   object        
dtypes: datetime64[ns](3), float64(1), object(5)
memory usage: 279.1+ MB


In [108]:
sub_cupones_totales=cupones_totales.drop_duplicates(['SUBSCRIPTION_CODE'], keep='last')

In [109]:
sub_cupones_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 41 to 3658469
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   user_id                       840 non-null    object        
 1   mark                          840 non-null    object        
 2   CLP_PRICE                     840 non-null    float64       
 3   SUBSCRIPTION_CODE             840 non-null    object        
 4   ACTIVE_SUB_END_DATE           840 non-null    datetime64[ns]
 5   ACTIVE_SUB_START_DATE         840 non-null    datetime64[ns]
 6   ACTIVE_SUB_LAST_PAYMENT_DATE  840 non-null    datetime64[ns]
 7   _id                           840 non-null    object        
 8   CUSTOMER_ID                   840 non-null    object        
dtypes: datetime64[ns](3), float64(1), object(5)
memory usage: 65.6+ KB


In [110]:
sub_user_Paypal=user_Paypal.drop_duplicates(['SUBSCRIPTION_CODE'], keep='last')

In [111]:
sub_user_Paypal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31690 entries, 0 to 31689
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   _id_x              31690 non-null  object        
 1   CUSTOMER_ID        31690 non-null  object        
 2   user_id_x          31690 non-null  object        
 3   id                 31690 non-null  object        
 4   SUBSCRIPTION_CODE  31690 non-null  object        
 5   createdAt          31690 non-null  datetime64[ns]
 6   user_id_y          31690 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 1.9+ MB


In [112]:
sub_user_transbank=user_transbank.drop_duplicates(['SUBSCRIPTION_CODE'], keep='last')

In [113]:
sub_user_transbank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135367 entries, 0 to 135366
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id                135367 non-null  object        
 1   CUSTOMER_ID        135367 non-null  object        
 2   user_id_x          135367 non-null  object        
 3   id                 135367 non-null  object        
 4   SUBSCRIPTION_CODE  135367 non-null  object        
 5   createdAt          135367 non-null  datetime64[ns]
 6   user_id_y          135367 non-null  object        
 7   START_DATE         135367 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 9.3+ MB


In [114]:
compendio=[sub_cupones_totales,sub_user_Paypal,sub_user_transbank]

In [115]:
universo=pd.concat(compendio)

In [116]:
SUBSCRIPTIONS_LIFETIME=universo.groupby(['CUSTOMER_ID']).agg(SUBSCRIPTIONS_LIFETIME=('SUBSCRIPTION_CODE','count'))

In [117]:
SUBSCRIPTIONS_LIFETIME['CUSTOMER_ID']=SUBSCRIPTIONS_LIFETIME.index

In [118]:
SUBSCRIPTIONS_LIFETIME=SUBSCRIPTIONS_LIFETIME.reset_index(drop=True)

In [119]:
conLifeTime=preresultado.merge(SUBSCRIPTIONS_LIFETIME, on='CUSTOMER_ID',how='outer')

In [120]:
conLifeTime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86586 entries, 0 to 86585
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   CUSTOMER_ID                   86586 non-null  object        
 1   SUBSCRIPTION_CODE             8664 non-null   object        
 2   CLP_PRICE                     8664 non-null   float64       
 3   ACTIVE_SUB_LAST_PAYMENT_DATE  8664 non-null   datetime64[ns]
 4   ACTIVE_SUB_END_DATE           8664 non-null   datetime64[ns]
 5   ACTIVE_SUB_START_DATE         8664 non-null   datetime64[ns]
 6   HAS_ACTIVE_SUBSCRIPTION       66416 non-null  float64       
 7   ACTIVE_SUB_DURATION           8664 non-null   float64       
 8   REVENUE_LIFETIME              66416 non-null  float64       
 9   SUBSCRIPTIONS_LIFETIME        80645 non-null  float64       
dtypes: datetime64[ns](3), float64(5), object(2)
memory usage: 7.3+ MB


### Ahora vamos por los acuerdos cancelados

In [121]:
feh

datetime.datetime(2017, 1, 1, 0, 0)

In [122]:
mycol_acuCancelados = mydb["acuerdoscancelados"]
def acuerdosCanceladosdataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
acuCancelados = acuerdosCanceladosdataframes(mycol_acuCancelados.find({'createdAt': {'$gte':feh}},{'acuerdo_id':1,'createdAt':1}), 10000)


In [123]:
acuCancelados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104875 entries, 0 to 4874
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   _id         104875 non-null  object        
 1   createdAt   104875 non-null  datetime64[ns]
 2   acuerdo_id  104875 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.2+ MB


In [124]:
acuCancelados['acuerdo_id']=acuCancelados.acuerdo_id.astype(str)

In [125]:
universo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167897 entries, 41 to 135366
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   user_id                       840 non-null     object        
 1   mark                          840 non-null     object        
 2   CLP_PRICE                     840 non-null     float64       
 3   SUBSCRIPTION_CODE             167897 non-null  object        
 4   ACTIVE_SUB_END_DATE           840 non-null     datetime64[ns]
 5   ACTIVE_SUB_START_DATE         840 non-null     datetime64[ns]
 6   ACTIVE_SUB_LAST_PAYMENT_DATE  840 non-null     datetime64[ns]
 7   _id                           136207 non-null  object        
 8   CUSTOMER_ID                   167897 non-null  object        
 9   _id_x                         31690 non-null   object        
 10  user_id_x                     167057 non-null  object        
 11  id          

In [126]:
universo.columns

Index(['user_id', 'mark', 'CLP_PRICE', 'SUBSCRIPTION_CODE',
       'ACTIVE_SUB_END_DATE', 'ACTIVE_SUB_START_DATE',
       'ACTIVE_SUB_LAST_PAYMENT_DATE', '_id', 'CUSTOMER_ID', '_id_x',
       'user_id_x', 'id', 'createdAt', 'user_id_y', 'START_DATE'],
      dtype='object')

In [127]:
universo=universo.drop(['user_id', 'mark', 'CLP_PRICE', 
       'ACTIVE_SUB_END_DATE', 'ACTIVE_SUB_START_DATE',
       'ACTIVE_SUB_LAST_PAYMENT_DATE', '_id',  '_id_x',
       'user_id_x', 'id', 'createdAt', 'user_id_y', 'START_DATE'],axis=1)

In [128]:
CANCELLATIONS_LIFETIME= acuCancelados.merge(universo, right_on='SUBSCRIPTION_CODE',left_on='acuerdo_id', how="inner", indicator=True)

In [129]:
CANCELLATIONS_LIFETIME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105473 entries, 0 to 105472
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id                105473 non-null  object        
 1   createdAt          105473 non-null  datetime64[ns]
 2   acuerdo_id         105473 non-null  object        
 3   SUBSCRIPTION_CODE  105473 non-null  object        
 4   CUSTOMER_ID        105473 non-null  object        
 5   _merge             105473 non-null  category      
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 4.9+ MB


In [131]:
CANCELLATIONS_LIFETIME = CANCELLATIONS_LIFETIME.dropna(axis=0, subset=['acuerdo_id'])

In [132]:
CANCELLATIONS_LIFETIME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105473 entries, 0 to 105472
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id                105473 non-null  object        
 1   createdAt          105473 non-null  datetime64[ns]
 2   acuerdo_id         105473 non-null  object        
 3   SUBSCRIPTION_CODE  105473 non-null  object        
 4   CUSTOMER_ID        105473 non-null  object        
 5   _merge             105473 non-null  category      
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 4.9+ MB


In [133]:
CANCELLATIONS=CANCELLATIONS_LIFETIME.groupby(['CUSTOMER_ID']).agg(CANCELLATIONS_LIFETIME=('acuerdo_id','count'))


In [134]:
CANCELLATIONS['CUSTOMER_ID']=CANCELLATIONS.index

In [135]:
CANCELLATIONS=CANCELLATIONS.reset_index(drop=True)


In [136]:
CANCELLATIONS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57644 entries, 0 to 57643
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CANCELLATIONS_LIFETIME  57644 non-null  int64 
 1   CUSTOMER_ID             57644 non-null  object
dtypes: int64(1), object(1)
memory usage: 900.8+ KB


In [137]:
conCancellations=conLifeTime.merge(CANCELLATIONS, on='CUSTOMER_ID',how='outer')

In [138]:
conCancellations["CANCELLATIONS_LIFETIME"] = conCancellations["CANCELLATIONS_LIFETIME"].fillna(0).astype(int)

In [139]:
conCancellations["HAS_ACTIVE_SUBSCRIPTION"] = conCancellations["HAS_ACTIVE_SUBSCRIPTION"].fillna(0).astype(int)

In [140]:
conCancellations["ACTIVE_SUB_DURATION"]=conCancellations["ACTIVE_SUB_DURATION"].fillna(0).astype(int)

In [141]:
conCancellations["SUBSCRIPTIONS_LIFETIME"]=conCancellations["SUBSCRIPTIONS_LIFETIME"].fillna(0).astype(int)

In [142]:
conCancellations["SUBSCRIPTIONS_LIFETIME"]=conCancellations["SUBSCRIPTIONS_LIFETIME"].fillna(0).astype(int)

In [143]:
conCancellations = conCancellations[['CUSTOMER_ID','ACTIVE_SUB_DURATION','ACTIVE_SUB_START_DATE','ACTIVE_SUB_LAST_PAYMENT_DATE','ACTIVE_SUB_END_DATE','HAS_ACTIVE_SUBSCRIPTION','CANCELLATIONS_LIFETIME','SUBSCRIPTIONS_LIFETIME','REVENUE_LIFETIME']]

In [144]:
import datetime
name='cdf_SubSummary'+datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
path='/home/tecnoboy/'+name+'.csv'
conCancellations.to_csv(path, index=False, quoting=csv.QUOTE_NONNUMERIC)